In [1]:
import time
time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(105484586906))

'Sat, 03 Sep 5312 10:08:26 +0000'

##Practical Example of BPNN

credit: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [2]:
from random import seed
from random import random

from math import exp
import numpy as np
import pandas as pd


In [3]:
 # Initialize a network, and randomly assign the weights
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
seed(10)
network = initialize_network(2, 1, 1)
for layer in network:
	print(layer)

[{'weights': [0.5714025946899135, 0.4288890546751146, 0.5780913011344704]}]
[{'weights': [0.20609823213950174, 0.81332125135732]}]


In [4]:
#manual assignment of weights
network = [[{'weights': [0.5, 0.1, 1]}],
 [{'weights': [1, -1]}]]

network

[[{'weights': [0.5, 0.1, 1]}], [{'weights': [1, -1]}]]

In [5]:
#Forward propagation


# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return np.round(1.0 / (1.0 + exp(-activation)),2)

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs) #linear combination
			neuron['output'] = transfer(activation) # activation function e.g. sigmoid
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# test forward propagation
row = [1, 2]
output = forward_propagate(network, row)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.1, 1], 'output': 0.85}]
[{'weights': [1, -1], 'output': 0.46}]


In [6]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# test backpropagation of error

expected = [1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.5, 0.1, 1], 'output': 0.85, 'delta': 0.017102340000000004}]
[{'weights': [1, -1], 'output': 0.46, 'delta': 0.13413600000000003}]


In [7]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']


lr =  .5
update_weights(network, row, lr)

for layer in network:
	print(layer)

[{'weights': [0.50855117, 0.1, 1.00855117], 'output': 0.85, 'delta': 0.017102340000000004}]
[{'weights': [1.0570078, -0.932932], 'output': 0.46, 'delta': 0.13413600000000003}]


In [8]:
#Apply this function to learn from data - you must provide the train variable which consist of your dataset

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row[:2])
      expected = row[2:3]
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
      # for layer in network:
        # print(layer)
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    


In [9]:
# Test training backprop algorithm
seed(1)

dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]


n_inputs = len(dataset[0]) - 1
n_hidden =  1
n_outputs = 1
network = initialize_network(n_inputs, n_hidden, n_outputs)

#use it if you want to determine initial weights:
# network = [[{'weights': [0.5, 0.1, 1]}],
#            [{'weights': [1, -1]}]]

epoch = 50
lr = 0.5
train_network(network, dataset, lr, epoch, n_outputs)
# for layer in network:
# 	print(layer)

>epoch=0, lrate=0.500, error=2.832
>epoch=1, lrate=0.500, error=2.740
>epoch=2, lrate=0.500, error=2.681
>epoch=3, lrate=0.500, error=2.672
>epoch=4, lrate=0.500, error=2.663
>epoch=5, lrate=0.500, error=2.632
>epoch=6, lrate=0.500, error=2.633
>epoch=7, lrate=0.500, error=2.640
>epoch=8, lrate=0.500, error=2.640
>epoch=9, lrate=0.500, error=2.629
>epoch=10, lrate=0.500, error=2.622
>epoch=11, lrate=0.500, error=2.611
>epoch=12, lrate=0.500, error=2.600
>epoch=13, lrate=0.500, error=2.590
>epoch=14, lrate=0.500, error=2.532
>epoch=15, lrate=0.500, error=2.459
>epoch=16, lrate=0.500, error=2.280
>epoch=17, lrate=0.500, error=2.121
>epoch=18, lrate=0.500, error=1.978
>epoch=19, lrate=0.500, error=1.843
>epoch=20, lrate=0.500, error=1.711
>epoch=21, lrate=0.500, error=1.599
>epoch=22, lrate=0.500, error=1.482
>epoch=23, lrate=0.500, error=1.372
>epoch=24, lrate=0.500, error=1.257
>epoch=25, lrate=0.500, error=1.181
>epoch=26, lrate=0.500, error=1.100
>epoch=27, lrate=0.500, error=1.009
>e

In [10]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs[0]

## Evaluation

In [11]:
actual  =  []
preds = []
for row in dataset:
  actual.append(row[-1])
  prediction = predict(network, row)
  preds.append(int(np.round(prediction)))
  print('Expected=%d, Got (actual)=%f, Got (prediction)=%d' % (row[-1], prediction, np.round(prediction)))

Expected=0, Got (actual)=0.190000, Got (prediction)=0
Expected=0, Got (actual)=0.140000, Got (prediction)=0
Expected=0, Got (actual)=0.130000, Got (prediction)=0
Expected=0, Got (actual)=0.160000, Got (prediction)=0
Expected=0, Got (actual)=0.170000, Got (prediction)=0
Expected=1, Got (actual)=0.830000, Got (prediction)=1
Expected=1, Got (actual)=0.800000, Got (prediction)=1
Expected=1, Got (actual)=0.830000, Got (prediction)=1
Expected=1, Got (actual)=0.830000, Got (prediction)=1
Expected=1, Got (actual)=0.820000, Got (prediction)=1


In [12]:
result  = pd.DataFrame((actual,preds)).T
result.columns  = ("actual", "preds")

In [13]:
conf_matrix  =  pd.crosstab(result.actual, result.preds)
conf_matrix

preds,0,1
actual,,
0,5,0
1,0,5


In [14]:
tn  = conf_matrix.iloc[0,0]
fp  = conf_matrix.iloc[0,1]
fn  = conf_matrix.iloc[1,0]
tp  = conf_matrix.iloc[1,1]

In [15]:
accuracy =  (tp+tn)/(tp+tn+fp+fn) * 100
recall =  tp/(tp+fn) * 100
precision  = tp/(tp+fp) * 100
f1Score = 2*(recall*precision)/(recall+precision)

In [16]:
print ('accuracy =  %.3f' % accuracy)
print ('recall =  %.3f'  % recall)
print ('precision =  %.3f' % precision)
print ('f1Score =  %.3f'  % f1Score)

accuracy =  100.000
recall =  100.000
precision =  100.000
f1Score =  100.000


1. Ubah online learning menjadi batch learning
2. Modifikasi train network sehingga bisa menggunakan val data (per 5 epoch)
3. Evaluation menggunakan test data (bandingkan dengan accuracy training, validation)